In [1]:
from __future__ import division
import datetime
import matplotlib.pyplot as plt
import cPickle as pickle
import numpy as np
import v2gsim
import pandas as pd

# ### Require gurobi or CPLEX #####
# Create a project and initialize it with some itineraries
project = v2gsim.model.Project()
project = v2gsim.itinerary.from_excel(project, '../../data/NHTS/California.xlsx')
project = v2gsim.itinerary.copy_append(project, nb_of_days_to_add=2)

# This function from the itinerary module return all the vehicles that
# start and end their day at the same location (e.g. home)
project.vehicles = v2gsim.itinerary.get_cycling_itineraries(project)


# Reduce the number of vehicles
project.vehicles = project.vehicles[0:100]

# Create some new charging infrastructures, append those new
# infrastructures to the project list of infrastructures
charging_stations = []
charging_stations.append(
    v2gsim.model.ChargingStation(name='L2', maximum_power=7200, minimum_power=0))
charging_stations.append(
    v2gsim.model.ChargingStation(name='L1_V1G', maximum_power=1400, minimum_power=0, post_simulation=True))
charging_stations.append(
    v2gsim.model.ChargingStation(name='L2_V2G', maximum_power=7200, minimum_power=-7200, post_simulation=True))
project.charging_stations.extend(charging_stations)

 #Create a data frame with the new infrastructures mix and
# apply this mix at all the locations
df = pd.DataFrame(index=['L2', 'L1_V1G', 'L2_V2G'],
                      data={'charging_station': charging_stations,
                            'probability': [0.0, 0.4, 0.6]})
for location in project.locations:
    if location.category in ['Work', 'Home']:
        location.available_charging_station = df.copy()

# Initiate SOC and charging infrastructures
v2gsim.core.initialize_SOC(project, nb_iteration=2)

# Assign a basic result function to save power demand
for vehicle in project.vehicles:
    vehicle.result_function = v2gsim.post_simulation.netload_optimization.save_vehicle_state_for_optimization


# Launch the simulation
v2gsim.core.run(project, date_from=project.date + datetime.timedelta(days=1),
                date_to=project.date + datetime.timedelta(days=2),
                reset_charging_station=False)


# Look at the results
total_power_demand = v2gsim.post_simulation.result.total_power_demand(project)

# Optimization
myopti = v2gsim.post_simulation.netload_optimization.CentralOptimization(project, 10,
                                                                         project.date + datetime.timedelta(days=1),
                                                                         project.date + datetime.timedelta(days=2),
                                                                         minimum_SOC=0.1, maximum_SOC=0.95)

# Load the net load data
finalResult = pd.DataFrame()
filename = '../../data/netload/2025.pickle'
# with open(filename, 'rb') as fp:
#     net_load = pickle.load(fp)
net_load=pd.read_pickle(filename)
day = datetime.datetime(2025, 6, 17)
net_load = pd.DataFrame(net_load[day: day + datetime.timedelta(days=1)]['netload'])
net_load.head()
net_load.shape

net_load.to_csv('../../UCED/netload.csv')

pr=pd.read_csv("../../UCED/out_camb_R1_2018_price.csv")
########## put into model already
myresult= myopti.solve(project, net_load * 1000000,1500000, price=pr, peak_shaving='economic', SOC_margin=0.05)

# total_power_demand['total']
# myresult,model=myresult 
# # myresult['vehicle_before'] 
# # myresult['vehicle_after']) *(1500000 / len(project.vehicles)) / (1000 * 1000) 
# myresult
myresult.to_csv('../../data/vehicle/VehicleCap.csv',header=0)


/Users/meiyewang/opt/anaconda3/envs/py27/lib/python2.7/site-packages/pandas/util/_decorators.py:188: FutureWarning: The `sheetname` keyword is deprecated, use `sheet_name` instead
  return func(*args, **kwargs)
/Users/meiyewang/opt/anaconda3/envs/py27/lib/python2.7/site-packages/v2gsim/core.py:194: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=False'.

To retain the current behavior and silence the warning, pass 'sort=True'.

  axis=0)
core.initialize_SOC: 100%|####################################################|
core.run:  26%|################                                               |       mean  mean_rate       std  std_rate
0  0.950000   0.000000  0.000000  0.000000
1  0.945551   0.004449  0.038983 -0.038983
2  0.945373   0.000178  0.038857  0.000126

core.run: 100%|###############################################################|core.run: 100%|####

In [57]:
# pd.DataFrame.from_dict(data=u1)
p1=pd.DataFrame(index=['energy'], data=u1).transpose()

p1_discharge=p1[p1['energy']>0]
p1_charge=-p1[p1['energy']<0]
p1_discharge=p1_discharge.reindex(p1.index,fill_value=0)
p1_discharge=p1_discharge.rename(columns={'energy':'discharge'})
p1_charge=p1_charge.reindex(p1.index,fill_value=0)
p1_charge=p1_charge.rename(columns={'energy':'charge'})

p2=pd.DataFrame(index=['energy binary'], data=ub).transpose()
p3=pd.DataFrame(index=['regup'], data=c1).transpose()
p4=pd.DataFrame(index=['regup binary'], data=cb).transpose()
p5=pd.DataFrame(index=['regdown'], data=c2).transpose()
p6=pd.DataFrame(index=['regdown binary'], data=cb2).transpose()

result_pd=pd.concat([p1_charge,p1_discharge,p2,p3,p4,p5,p6],axis=1)
result_pd['EnergyDemand']=result_pd['charge']*result_pd['energy binary']
result_pd['EnergyGeneration']=result_pd['discharge']*result_pd['energy binary']
result_pd['Regup']=result_pd['regup']*result_pd['regup binary']
result_pd['Regdown']=result_pd['regdown']*result_pd['regdown binary']
result_pd=result_pd.drop(['discharge','charge','energy binary','regup','regup binary','regdown','regdown binary'],axis=1).groupby(level=0).sum()

In [59]:
result_pd.set_index(i)

KeyError: 0

In [60]:
i

0

In [43]:
p2=pd.DataFrame(index=['energy binary'], data=ub).transpose()
p3=pd.DataFrame(index=['regup'], data=c1).transpose()
p4=pd.DataFrame(index=['regup binary'], data=cb).transpose()
p5=pd.DataFrame(index=['regdown'], data=c2).transpose()
p6=pd.DataFrame(index=['regdown binary'], data=cb2).transpose()

result_pd=pd.concat([p1,p2,p3,p4,p5,p6],axis=1)
result_pd['Energy']=result_pd['energy']*result_pd['energy binary']
result_pd['Regup']=result_pd['regup']*result_pd['regup binary']
result_pd['Regdown']=result_pd['regdown']*result_pd['regdown binary']
result_pd=result_pd.drop(['energy','energy binary','regup','regup binary','regdown','regdown binary'],axis=1).groupby(level=0).sum()

In [23]:
d = u1
type(d)
# d=d.update(ub)
# d=d.update(c1)
# d=d.update(cb)
# d=d.update(cb2)
# rawData=pd.DataFrame.from_dict(data=d,orient='index')
# rawData


dict

In [3]:
myresult,model=myresult

ValueError: too many values to unpack

In [4]:
df,power,powerresult,powerresult_pd,i=myresult

In [6]:
power

,power discharged,power charged,power discharged binary,power regulation up,power regulation up binary,power regulation down,power regulation down binary
0,0.0,6600.0,0.0,0.0,0.0,464800.0,98.0
1,0.0,6600.0,0.0,0.0,0.0,464800.0,98.0
2,0.0,6600.0,0.0,0.0,0.0,464800.0,98.0
3,0.0,6600.0,0.0,0.0,0.0,464800.0,98.0
4,0.0,6600.0,0.0,0.0,0.0,464800.0,98.0
5,0.0,6600.0,0.0,0.0,0.0,464800.0,98.0
6,0.0,6600.0,0.0,1400.0,1.0,463400.0,97.0
7,0.0,6600.0,0.0,202800.0,52.0,262000.0,46.0
8,0.0,6600.0,0.0,407400.0,83.0,57400.0,15.0
9,0.0,6600.0,0.0,355600.0,72.0,109200.0,26.0


In [7]:
powerresult

 float64, 20    6600.0
  dtype: float64, 20    19140000.0
  dtype: float64, 20    5593600.0
  dtype: float64, 20   -6600.0
  dtype: float64), (21    0.0
  dtype: float64, 21    6600.0
  dtype: float64, 21    9210600.0
  dtype: float64, 21    13357400.0
  dtype: float64, 21   -6600.0
  dtype: float64), (22    0.0
  dtype: float64, 22    6600.0
  dtype: float64, 22    19872200.0
  dtype: float64, 22    5214200.0
  dtype: float64, 22   -6600.0
  dtype: float64), (23    0.0
  dtype: float64, 23    6600.0
  dtype: float64, 23    0.0
  dtype: float64, 23    45550400.0
  dtype: float64, 23   -6600.0
  dtype: float64), (24    0.0
  dtype: float64, 24    6600.0
  dtype: float64, 24    1400.0
  dtype: float64, 24    43852800.0
  dtype: float64, 24   -6600.0
  dtype: float64), (25    0.0
  dtype: float64, 25    6600.0
  dtype: float64, 25    1400.0
  dtype: float64, 25    43852800.0
  dtype: float64, 25   -6600.0
  dtype: float64), (26    0.0
  dtype: float64, 26    6600.0
  dtype: float64, 26   

In [8]:
model

NameError: name 'model' is not defined

In [9]:
powerresult_pd

,Discharge,Charge,Regup,Regdown,EnergySum
0,0 0.0 dtype: float64,0 6600.0 dtype: float64,0 0.0 dtype: float64,0 45550400.0 dtype: float64,0 -6600.0 dtype: float64
1,1 0.0 dtype: float64,1 6600.0 dtype: float64,1 0.0 dtype: float64,1 45550400.0 dtype: float64,1 -6600.0 dtype: float64
2,2 0.0 dtype: float64,2 6600.0 dtype: float64,2 0.0 dtype: float64,2 45550400.0 dtype: float64,2 -6600.0 dtype: float64
3,3 0.0 dtype: float64,3 6600.0 dtype: float64,3 0.0 dtype: float64,3 45550400.0 dtype: float64,3 -6600.0 dtype: float64
4,4 0.0 dtype: float64,4 6600.0 dtype: float64,4 0.0 dtype: float64,4 45550400.0 dtype: float64,4 -6600.0 dtype: float64
5,5 0.0 dtype: float64,5 6600.0 dtype: float64,5 0.0 dtype: float64,5 45550400.0 dtype: float64,5 -6600.0 dtype: float64
6,6 0.0 dtype: float64,6 6600.0 dtype: float64,6 1400.0 dtype: float64,6 44949800.0 dtype: float64,6 -6600.0 dtype: float64
7,7 0.0 dtype: float64,7 6600.0 dtype: float64,7 10545600.0 dtype: float64,7 12052000.0 dtype: float64,7 -6600.0 dtype: float64
8,8 0.0 dtype: float64,8 6600.0 dtype: float64,8 33814200.0 dtype: float64,8 861000.0 dtype: float64,8 -6600.0 dtype: float64
9,9 0.0 dtype: float64,9 6600.0 dtype: float64,9 25603200.0 dtype: float64,9 2839200.0 dtype: float64,9 -6600.0 dtype: float64


In [10]:
power.head(144)

,power discharged,power charged,power discharged binary,power regulation up,power regulation up binary,power regulation down,power regulation down binary
0,0.0,6600.0,0.0,0.000000,0.0,464800.0,98.0
1,0.0,6600.0,0.0,0.000000,0.0,464800.0,98.0
2,0.0,6600.0,0.0,0.000000,0.0,464800.0,98.0
3,0.0,6600.0,0.0,0.000000,0.0,464800.0,98.0
4,0.0,6600.0,0.0,0.000000,0.0,464800.0,98.0
5,0.0,6600.0,0.0,0.000000,0.0,464800.0,98.0
6,0.0,6600.0,0.0,1400.000000,1.0,463400.0,97.0
7,0.0,6600.0,0.0,202800.000000,52.0,262000.0,46.0
8,0.0,6600.0,0.0,407400.000000,83.0,57400.0,15.0
9,0.0,6600.0,0.0,355600.000000,72.0,109200.0,26.0


In [12]:
import pandas 
powerresult =[]

dischar=[]
char=[]
regup=[]
regdown=[]
powersum=[]

for i in range(len(df)):           
    dischar.append(power.loc[[i],'power discharged']*power.loc[[i],'power discharged binary'])
    char.append(power.loc[[i],'power charged']*power.loc[[i],'power discharged binary'])
    regup.append(power.loc[[i],'power regulation up']*power.loc[[i],'power regulation up binary'])
    regdown.append(power.loc[[i],'power regulation down']*power.loc[[i],'power regulation down binary'])
    powersum.append(dischar[i]-char[i])
    powerresult.append((dischar[i],char[i],regup[i],regdown[i],powersum[i]))

powerresult_pd=pandas.DataFrame(powerresult,columns=('Discharge','Charge','Regup','Regdown','EnergySum'))


In [16]:
powerresult=power
powerresult['Discharge']=powerresult['power discharged']*powerresult['power discharged binary']
powerresult['Charge']=powerresult['power charged']*(1-powerresult['power discharged binary'])


SyntaxError: invalid syntax (<ipython-input-16-4b88567422ed>, line 3)

In [23]:
power['Discharge']=power['power discharged']*power['power discharged binary']
power['Charge']=power['power charged']*(1-power['power discharged binary'])
power['Regup']=power['power regulation up']*power['power regulation up binary']
power['Regdown']=power['power regulation down']*power['power regulation down binary']
power.drop(columns=['power discharged','power charged','power discharged binary','power regulation up','power regulation up binary','power regulation down','power regulation down binary'],axis=1).loc[0:143]
power = power.set_index(i)

KeyError: 143

In [26]:
df

,power regulation down binary
0,98.0
1,98.0
2,98.0
3,98.0
4,98.0
5,98.0
6,97.0
7,46.0
8,15.0
9,26.0
